# Gradient Checking

jhbale11

[Coursera]Andrew Ng 's Deep Learning Lecture

https://github.com/suqi/deeplearning_andrewng

앤드류 응 교수님의 딥러닝 강의를 듣고 예제를 제 방식대로 연습한 결과입니다.

In [1]:
# Packages
import numpy as np
from utils_6 import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 1) How does gradient checking work?

역전파는 gradients를 계산할 것이며, gradients의 요소들은 순전파를 통해 구해집니다.

순전파는 비교적 구현이 쉽기 때문에, 역전파에 비해 올바르게 구현했을 가능성이 높습니다. 따라서 순전파만을 이용해 역전파를 근사할 수 있다면 역전파를 제대로 구현했는지 좀 더 확신할 수 있을 것입니다. 

다음 공식을 수치적으로 계산하면 순전파를 이용해 기울기를 근사적으로 구할 수 있습니다.
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

## 2) 1-dimensional gradient checking

우선 쉬운 예제로서 $J(\theta) = \theta x$에 대해 gradient checking을 해볼 것입니다.

In [2]:
def forward_propagation(x, theta):
    J = theta * x
    return J

def backward_propagation(x, theta):
    dtheta = x
    return dtheta

- 위에 있는 (1) 공식을 이용해 $\theta$에 대한 미분값을 근사적으로 계산하고, 이를 역전파를 통해 구한 값과 비교해야 합니다. 두 값의 차이는 다음 공식을 이용해 계산합니다.

- $\theta^{+} = \theta + \varepsilon$
- $\theta^{-} = \theta - \varepsilon$
- $J^{+} = J(\theta^{+})$
- $J^{-} = J(\theta^{-})$
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$
- 만약 차이가 작다면 (가령 $10^{-7}$보다 작다면), 역전파를 올바르게 구현했다고 자신해도 좋습니다! 


In [3]:
def gradient_check(x, theta, epsilon = 1e-7):
    theta_plus = theta + epsilon
    theta_minus = theta - epsilon
    J_plus = forward_propagation(x, theta_plus)
    J_minus = forward_propagation(x, theta_minus)
    
    gradapprox = (J_plus - J_minus) / (2*epsilon)
    grad = backward_propagation(x, theta)
    
    difference = np.linalg.norm(grad - gradapprox) / (np.linalg.norm(grad) + np.linalg.norm(gradapprox))
    
    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

In [4]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.919335883291695e-10


## 3) N-dimensional gradient checking

이제 좀 더 일반화된 경우를 연습하기 위해 3층 신경망에 대한 gradient checking을 구현해보겠습니다. 

이번에 사용할 DNN은 다음과 같이 구성되어 있습니다.
1) LINEAR -> RELU
2) LINEAR -> RELU
3) LINEAR -> SIGMOID

In [5]:
def forward_propagation_n(X, Y, parameters):
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cost = -np.sum(Y * np.log(A3) + (1 - Y) * np.log(1-A3)) / m
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

def backward_propagation_n(X, Y, cache):
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = np.dot(dZ3, A2.T) / m
    db3 = np.sum(dZ3, axis=1, keepdims=True) / m
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = dA2 * (Z2>0)
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * (Z1>0)
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
        
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

이전과 마찬가지 방식으로 그레이언트 검사를 수행합니다. 

단 이번에는 검사해야 할 파라미터의 개수가 많습니다. 각각의 파라미터 모두에 대해 위에서 했던 방식을 반복하기 위해서, 모든 파라미터를 하나의 백터로 만들어서 계산하는 방식을 사용해보겠습니다. 이를 위해 사전에 정의된 `dictionary_to_vector` 등의 함수를 이용합니다 dictionary_to_vector 함수는 Utils 파일에 정의해두었습니다.

In [6]:
def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    
    # 모든 파라미터("W1", "b1", "W2", "b2", "W3", "b3")를 (-1, 1) 크기를 가진 하나의 벡터로 만든다.
    parameters_values, _ = dictionary_to_vector(parameters)
    
    # 모든 그레디언트(backward_propagation_n의 반환값)를 (-1, 1) 크기를 가진 하나의 벡터로 만든다.
    grad = gradients_to_vector(gradients)
    
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # gradapprox 계산하기
    for i in range(num_parameters):
        theta_plus = np.copy(parameters_values)
        theta_plus[i, 0] += epsilon
        J_plus, _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_plus))

        theta_minus = np.copy(parameters_values)
        theta_minus[i, 0] -= epsilon
        J_minus, _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_minus))
        gradapprox[i] = (J_plus - J_minus) / (2*epsilon)
        
    difference = np.linalg.norm(grad - gradapprox) / (np.linalg.norm(grad)+np.linalg.norm(gradapprox))

    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [7]:
# 잘 작동하는지 확인하기 위해 임의의 값들을 넣어서 확인.
def gradient_check_n_test_case(): 
    x = np.random.randn(4,3)
    y = np.array([1, 1, 0])
    W1 = np.random.randn(5,4) 
    b1 = np.random.randn(5,1) 
    W2 = np.random.randn(3,5) 
    b2 = np.random.randn(3,1) 
    W3 = np.random.randn(1,3) 
    b3 = np.random.randn(1,1) 
    parameters = {"W1": W1,"b1": b1,"W2": W2,"b2": b2,"W3": W3,"b3": b3}
    return x, y, parameters

X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

Your backward propagation works perfectly fine! difference = 2.989007775278216e-09


- 그레디언트를 수치적으로 계산하는 것은 느리기 때문에 코드가 제대로 짜여졌는지 확인하는 용도로만 사용하는 것이 좋습니다.
- 드롭아웃을 사용할 경우, 우선 드롭아웃이 없는 채로 그레디언트 검증을 사용하고 드롭아웃을 추가하여야합니다.